In [1]:
# Corrección de cuantiles empírica.

import numpy as np
import pandas as pd

import xarray as xr

In [2]:
v = "Pcp"
dims = ["XTIME", "XLAT", "XLONG"]

path_d    = "../../temp/cc_idt/era5-land_total-precipitation_cdf.nc"
path_m    = "../temp/WRF_regrid_CHIRPS_days_cdf.nc"
path_f    = "../temp/WRF_2040_regrid_CHIRPS_days_cdf.nc"
path_res  = "../temp/WRF_regrid_CHIRPS_days_map.nc"
path_map  = "../temp/WRF_regrid_CHIRPS_days_qmap.nc"
path_fmap = "../temp/WRF_2040_regrid_CHIRPS_days_qmap.nc"

#path_d    = "../temp/CHIRPS_megalopolis_cdf.nc"
#path_m    = "../temp/WRF_regrid_CHIRPS_days_cdf.nc"
#path_f    = "../temp/WRF_2040_regrid_CHIRPS_days_cdf.nc"
#path_res  = "../temp/WRF_regrid_CHIRPS_days_map.nc"
#path_map  = "../temp/WRF_regrid_CHIRPS_days_qmap.nc"
#path_fmap = "../temp/WRF_2040_regrid_CHIRPS_days_qmap.nc"

sum = True

with xr.open_dataset(path_m) as ds_m:
    with xr.open_dataset(path_d) as ds_d:
        with xr.open_dataset(path_f) as ds_f:

            df_d = ds_d.to_dataframe().sort_index()
            df_m = ds_m.to_dataframe().sort_index()
            df_f = ds_f.to_dataframe().sort_index()
            df_f["map"] = None
            df_m["map"] = None
            df_m["diff_sum"] = None
            #df_m["diff_div"] = None

            latitude = df_d.index.get_level_values(dims[1]).unique()
            longitude = df_d.index.get_level_values(dims[2]).unique()

            for lat in latitude:
                print(f"{lat:.3f}°N")
                for lon in longitude:
                    df_xs_m = df_m.loc[ (slice(None), lat, lon),
                        df_m.columns ].sort_values("q_model")
                    df_xs_d = df_d.loc[ (slice(None), lat, lon),
                        df_d.columns ].sort_values("q_model")
                    df_xs_m["map"] = np.interp( df_xs_m["q_model"].values,
                        df_xs_d["q_model"].values, df_xs_d[v].values )
                    df_xs_m["map"] = df_xs_m["map"].where(df_xs_m["map"]>0, 0)
                    df_xs_m["diff_sum"] = df_xs_m["map"] - df_xs_m[v]
                    df_m.loc[ (slice(None), lat, lon), df_m.columns ] = df_xs_m
                    #df_xs_m["diff_div"] = df_xs_m["map"] / df_xs_m[v]

                    df_xs_f = df_f.loc[ (slice(None), lat, lon), 
                        df_f.columns ].sort_values("q_model")
                    #if sum:
                    df_xs_f["map"] = np.interp( df_xs_f["q_model"].values,
                        df_xs_m["q_model"].values,
                        df_xs_m["diff_sum"].values )
                    df_xs_f["map"] = df_xs_f["map"] + df_xs_f[v]
                    #else:
                    #    df_xs_f["map"] = np.interp( df_xs_f["q_model"].values,
                    #        df_xs_m["q_model"].values,
                    #        df_xs_m["diff_div"].values )
                    #    df_xs_f["map"] = df_xs_f["map"] * df_xs_f[v]
                    df_xs_f["map"] = df_xs_f["map"].where(df_xs_f["map"]>0, 0)
                    df_f.loc[ (slice(None), lat, lon), df_f.columns ] = df_xs_f

            ds_map = df_m.to_xarray()
            ds_m["map"] = ( dims, ds_map["map"].values )
            ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
            #ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )
            ds_m.to_netcdf( path_res )
            ds_m[["map"]].rename( {"map": v} ).to_netcdf( path_map )

            ds_fmap = df_f.to_xarray()
            ds_f["map"] = ( dims, ds_fmap["map"].values )
            ds_f[["map"]].rename( {"map": v} ).to_netcdf( path_fmap )

18.125°N
18.175°N
18.225°N
18.275°N
18.325°N
18.375°N
18.425°N
18.475°N
18.525°N
18.575°N
18.625°N
18.675°N
18.725°N
18.775°N
18.825°N
18.875°N
18.925°N
18.975°N
19.025°N
19.075°N
19.125°N
19.175°N
19.225°N
19.275°N
19.325°N
19.375°N
19.425°N
19.475°N
19.525°N
19.575°N
19.625°N
19.675°N
19.725°N
19.775°N
19.825°N
19.875°N
19.925°N
19.975°N
20.025°N
20.075°N
20.125°N
20.175°N
20.225°N
20.275°N
20.325°N
20.375°N
20.425°N
20.475°N
20.525°N
20.575°N
20.625°N
20.675°N


KeyError: "No variable named 'diff_div'. Variables on the dataset include ['XTIME', 'XLAT', 'XLONG', 'Pcp', 'q_model', 'map', 'diff_sum']"